# init

In [1]:
import pandas, IPython.core.display

## open file

In [2]:
src = pandas.ExcelFile('data/real/2017/CAM data from iPads/2017 CAM data from iPads.xlsx')

In [3]:
src.sheet_names

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

## select sheets

In [4]:
sheets_to_process = {sheet_name.split(' ')[-1]: src.parse(sheet_name) for sheet_name in src.sheet_names
    if any([person_name in sheet_name for person_name in ['Erl', 'Tyler']])}

# rename columns

In [5]:
sheets_to_rename = [sheet for sheet in sheets_to_process.values()
                    if any([str(column).find(' ') for column in sheet.columns])]

In [6]:
for sheet_to_rename in sheets_to_rename:
    sheet_to_rename.rename(mapper=lambda x: str(x).split(' ')[0], axis='columns', inplace=True)

# concatenate sheets

In [7]:
df = pandas.concat(sheets_to_process, names=['Sheet','Row'])

# hierarchical index

In [8]:
import collections

In [9]:
class NestedDefaultdict(collections.defaultdict):
    data = None
    name = None
    parent_key = None

    def __init__(self):
        collections.defaultdict.__init__(self, NestedDefaultdict)

In [10]:
def get_column_tree(frame):
    coltree = NestedDefaultdict()
    for column in frame.columns:
        pointer = coltree
        pointer.parent_key = ''
        for word in str(column).split('__'):
            pointer = pointer[word]
            pointer.parent_key = word
        pointer.name = column
        pointer.data = frame[column]
    coltree.default_factory = None
    return coltree

In [21]:
coltree = get_column_tree(df)

## visualize

In [11]:
def visualize_tree(node):
    """Given a dictionary, display a list of keys, and all keys 
    of any contained dictionaries, recursively. Indent to show depth
    of recursion."""

    def visualize_node(node, depth=0):
        for key, value in node.items():
            output.append(f"{'.   ' * depth}{key}"
                          f"{'/' if value.keys() else ''}"
                          f"{' : ' + str(value.data.count()) if value.data is not None else ''}"
                         )
            visualize_node(value, depth + 1)

    output = []
    visualize_node(node)
    return '\n'.join(output)

# define functions: get nodes, column names, DataFrames

In [15]:
def get_child_columns(node):
    return_value = []
    if node.data is not None:
        return_value.append(node)
    for child in node.values():
        return_value.append(child)
    return return_value

In [12]:
def get_descendant_columns(node):
    return_value = []
    if node.data is not None:
        return_value.append(node)
    for child in node.values():
        return_value.extend(get_descendant_columns(child))
    return return_value

In [16]:
def get_child_column_names(node):
    return [column.name for column in get_child_columns(node)]

In [13]:
def get_descendant_column_names(node):
    return [column.name for column in get_descendant_columns(node)]

In [17]:
def get_child_frame(node, frame=None):
    if frame is None:
        return pandas.DataFrame({column.name: column.data for column in get_child_columns(node)})
    return frame[get_child_column_names(node)]

In [14]:
def get_descendant_frame(node, frame=None):
    if frame is None:
        return pandas.DataFrame({column.name: column.data for column in get_descendant_columns(node)})
    return frame[get_descendant_column_names(node)]

# convert_datetime

In [18]:
def convert_datetime(frame):
    output_frame = frame.copy()
    foplt = 'fields__oSets__oPoints__location__timestamp'
    output_frame[foplt] = pandas.to_datetime(frame[foplt], unit='ms')
    for column in ['fields__oSets__date', 'fields__date']:
        output_frame[column] = pandas.to_datetime(output_frame[column], infer_datetime_format=True)
    return output_frame

In [19]:
df = convert_datetime(df)

# records: fields

In [22]:
get_child_frame(coltree['fields']).dropna(how='all')

NaN  fields__crop            fields__date  \
Sheet Row                                                
Erl   70    None   CROPS.WHEAT 2017-08-09 09:24:11.845   
      140   None   CROPS.WHEAT 2017-08-09 10:01:29.326   
      210   None  CROPS.BARLEY 2017-08-09 11:16:15.922   
      350   None    CROPS.OATS 2017-08-09 11:17:15.791   
      490   None   CROPS.WHEAT 2017-08-10 14:27:04.887   
      560   None   CROPS.WHEAT 2017-08-17 11:10:41.881   
      700   None   CROPS.WHEAT 2017-08-17 13:05:51.122   
      840   None   CROPS.WHEAT 2017-08-22 16:01:41.018   
Tyler 0     None   CROPS.WHEAT 2017-07-14 12:30:31.587   
      350   None  CROPS.BARLEY 2017-07-18 10:46:49.163   
      770   None    CROPS.OATS 2017-07-18 11:35:17.050   
      1190  None   CROPS.WHEAT 2017-07-18 14:07:55.743   
      1610  None  CROPS.BARLEY 2017-07-18 14:39:49.012   
      2030  None  CROPS.BARLEY 2017-07-19 10:21:45.036   
      2240  None   CROPS.WHEAT 2017-07-19 10:41:49.541   
      2520  None   CROPS.WHEAT 2017-07-19 14:07:06.447   
      2870  None   CROPS.WHEAT 2017-08-01 11:35:22.572   
      3080  None   CROPS.WHEAT 2017-08-01 12:59:56.247   
      3290  None   CROPS.WHEAT 2017-08-10 14:21:41.648   
      3640  None   CROPS.WHEAT 2017-08-10 14:36:39.936   
      3710  None   CROPS.WHEAT 2017-08-22 15:41:43.544   

                                                 fields__desc   fields__image  \
Sheet Row                                                                       
Erl   70                      Midge susceptible wheat. Awned.   img/wheat.png   
      140               Early planted awned midge susceptible   img/wheat.png   
      210              Next to SEF Wheat and near faba beans   img/barley.png   
      350                  Next to SEF peas and a wheat field    img/oats.png   
      490                                   Kernen cover crop   img/wheat.png   
      560   Wheat 2 by canola and soybeans but on other si...   img/wheat.png   
      700                         Next to soybeans and canola   img/wheat.png   
      840                                                 NaN   img/wheat.png   
Tyler 0                                                   NaN   img/wheat.png   
      350                             Right next to SEF wheat  img/barley.png   
      770                  Next to SEF Peas, near the canola     img/oats.png   
      1190                                                NaN   img/wheat.png   
      1610                                                NaN  img/barley.png   
      2030                        Right next to Alvena Canola  img/barley.png   
      2240                 Futher down the road towards Wakaw   img/wheat.png   
      2520                                   Next to treeline   img/wheat.png   
      2870                                                NaN   img/wheat.png   
      3080   Wheat 1 is next to the faba beans and soybeans I   img/wheat.png   
      3290          Research field cover crop. Student trial.   img/wheat.png   
      3640                                                NaN   img/wheat.png   
      3710                                Closer to compound    img/wheat.png   

                  fields__name  
Sheet Row                       
Erl   70     Llewellyn wheat 1  
      140            Llewellyn  
      210           SEF Barley  
      350             SEF Oats  
      490         Kernen wheat  
      560        Outlook wheat  
      700      Outlook wheat 1  
      840       Kernan Wheat-1  
Tyler 0              Sef wheat  
      350          SEF Barley   
      770             SEF Oats  
      1190    Llewelyn Wheat-1  
      1610     Llewelyn Barley  
      2030       Alvena Barley  
      2240        Alvena Wheat  
      2520       Melfort Wheat  
      2870     Outlook Wheat-2  
      3080     Outlook Wheat-1  
      3290        Kernen wheat  
      3640  Kernen cover wheat  
      3710      Kernan wheat 2

# records: sets

In [23]:
get_child_frame(coltree['fields']['oSets']).dropna(how='all')

fields__oSets__completeSets     fields__oSets__date  \
Sheet Row                                                         
Erl   0                             0.0 2017-08-02 13:12:09.542   
      70                            1.0 2017-08-09 09:25:11.710   
      140                           1.0 2017-08-09 10:06:25.480   
      210                           2.0 2017-08-09 11:21:01.555   
      350                           2.0 2017-08-09 11:37:20.862   
      490                           1.0 2017-08-22 15:42:05.751   
      560                           2.0 2017-08-17 11:12:02.820   
      700                           2.0 2017-08-17 13:06:30.183   
      840                           1.0 2017-08-22 16:02:50.682   
Tyler 0                             0.0 2017-07-14 12:31:24.194   
      70                            1.0 2017-07-18 10:31:22.263   
      140                           1.0 2017-07-28 13:05:44.673   
      210                           1.0 2017-08-01 15:27:40.174   
      280                           1.0 2017-08-04 10:18:29.654   
      350                           1.0 2017-07-18 10:51:30.195   
      420                           1.0 2017-07-28 13:20:31.265   
      490                           1.0 2017-08-01 15:37:15.582   
      560                           1.0 2017-08-04 10:25:17.707   
      630                           2.0 2017-08-18 10:59:51.840   
      770                           1.0 2017-07-18 11:39:46.221   
      840                           1.0 2017-07-28 13:32:37.048   
      910                           1.0 2017-08-01 15:47:22.848   
      980                           1.0 2017-08-04 10:33:20.587   
      1050                          2.0 2017-08-18 11:13:51.831   
      1190                          1.0 2017-07-18 14:25:46.769   
      1260                          1.0 2017-07-28 14:53:59.672   
      1330                          1.0 2017-08-01 16:18:37.497   
      1400                          1.0 2017-08-03 11:10:21.408   
      1470                          2.0 2017-08-18 14:16:58.657   
      1610                          1.0 2017-07-18 14:49:08.831   
      1680                          1.0 2017-07-28 14:34:36.584   
      1750                          1.0 2017-08-01 16:16:07.025   
      1820                          1.0 2017-08-03 11:28:09.304   
      1890                          2.0 2017-08-18 13:33:03.584   
      2030                          1.0 2017-07-19 10:22:29.528   
      2100                          1.0 2017-07-28 11:07:21.041   
      2170                          1.0 2017-08-02 15:06:10.327   
      2240                          1.0 2017-07-19 11:22:41.376   
      2310                          1.0 2017-07-28 10:13:55.165   
      2380                          1.0 2017-08-02 14:28:33.420   
      2450                          1.0 2017-08-24 16:57:43.540   
      2520                          1.0 2017-07-19 14:07:39.723   
      2590                          1.0 2017-07-26 12:23:53.773   
      2660                          1.0 2017-08-02 11:43:20.708   
      2730                          2.0 2017-08-24 13:10:20.476   
      2870                          1.0 2017-08-01 11:36:00.012   
      2940                          2.0 2017-08-25 12:54:07.934   
      3080                          1.0 2017-08-01 13:01:17.968   
      3150                          2.0 2017-08-25 13:05:14.062   
      3290                          0.0 2017-08-10 14:22:57.492   
      3360                          2.0 2017-08-18 15:31:02.572   
      3500                          2.0 2017-08-22 16:01:49.995   
      3640                          1.0 2017-08-10 14:37:29.806   
      3710                          1.0 2017-08-22 15:42:36.395   

           fields__oSets__dateCompare  fields__oSets__desc  \
Sheet Row                                                    
Erl   0                    2017-08-02                  NaN   
      70                   2017-08-09                  NaN   
      140  

# records: points

In [24]:
get_child_frame(coltree['fields']['oSets']['oPoints']).dropna(how='all')

fields__oSets__oPoints__id   NaN fields__oSets__oPoints__name
Sheet Row                                                                
Erl   0                            0.0  None          Observation Point 1
      14                           1.0  None          Observation Point 2
      28                           2.0  None          Observation Point 3
      42                           3.0  None          Observation Point 4
      56                           4.0  None          Observation Point 5
      70                           0.0  None          Observation Point 1
      84                           1.0  None          Observation Point 2
      98                           2.0  None          Observation Point 3
      112                          3.0  None          Observation Point 4
      126                          4.0  None          Observation Point 5
      140                          0.0  None          Observation Point 1
      154                          1.0  None          Observation Point 2
      168                          2.0  None          Observation Point 3
      182                          3.0  None          Observation Point 4
      196                          4.0  None          Observation Point 5
      210                          0.0  None          Observation Point 1
      224                          1.0  None          Observation Point 2
      238                          2.0  None          Observation Point 3
      252                          3.0  None          Observation Point 4
      266                          4.0  None          Observation Point 5
      280                          5.0  None          Observation Point 6
      294                          6.0  None          Observation Point 7
      308                          7.0  None          Observation Point 8
      322                          8.0  None          Observation Point 9
      336                          9.0  None         Observation Point 10
      350                          0.0  None          Observation Point 1
      364                          1.0  None          Observation Point 2
      378                          2.0  None          Observation Point 3
      392                          3.0  None          Observation Point 4
      406                          4.0  None          Observation Point 5
...                                ...   ...                          ...
Tyler 3360                         0.0  None          Observation Point 1
      3374                         1.0  None          Observation Point 2
      3388                         2.0  None          Observation Point 3
      3402                         3.0  None          Observation Point 4
      3416                         4.0  None          Observation Point 5
      3430                         5.0  None          Observation Point 6
      3444                         6.0  None          Observation Point 7
      3458                         7.0  None          Observation Point 8
      3472                         8.0  None          Observation Point 9
      3486                         9.0  None         Observation Point 10
      3500                         0.0  None          Observation Point 1
      3514                         1.0  None          Observation Point 2
      3528                         2.0  None          Observation Point 3
      3542                         3.0  None          Observation Point 4
      3556                         4.0  None          Observation Point 5
      3570                         5.0  None          Observation Point 6
      3584                         6.0  None          Observation Point 7
      3598                         7.0  None          Observation Point 8
      3612                         8.0  None          Observation Point 9
      3626                         9.0  None         Observation Point 10
      3640                         0.0  None          Observation Point 1
      3654      

# records: observations

In [25]:
get_descendant_frame(coltree['fields']['oSets']['oPoints']['observations']).dropna(how='all')

fields__oSets__oPoints__observations__a1__number  \
Sheet Row                                                      
Erl   0                                                  NaN   
      1                                                  NaN   
      2                                                  NaN   
      3                                                  NaN   
      4                                                  NaN   
      5                                                  NaN   
      6                                                  NaN   
      7                                                  NaN   
      8                                                  NaN   
      9                                                  NaN   
      10                                                 NaN   
      11                                                 NaN   
      12                                                 NaN   
      13                                                 NaN   
      14                                                 NaN   
      15                                                 NaN   
      16                                                 NaN   
      17                                                 NaN   
      18                                                 NaN   
      19                                                 NaN   
      20                                                 NaN   
      21                                                 NaN   
      22                                                 NaN   
      23                                                 NaN   
      24                                                 NaN   
      25                                                 NaN   
      26                                                 NaN   
      27                                                 NaN   
      28                                                 NaN   
      29                                                 NaN   
...                                                      ...   
Tyler 3750                                               NaN   
      3751                                               NaN   
      3752                                               1.0   
      3753                                               NaN   
      3754                                               NaN   
      3755                                               NaN   
      3756                                               1.0   
      3757                                               NaN   
      3758                                               NaN   
      3759                                               NaN   
      3760                                               NaN   
      3761                                               NaN   
      3762                                               NaN   
      3763                                               NaN   
      3764                                               NaN   
      3765                                               NaN   
      3766                                               2.0   
      3767                                               NaN   
      3768                                               NaN   
      3769                                               NaN   
      3770                                               NaN   
      3771                                               NaN   
      3772                                               NaN   
      3773                                               NaN   
      3774                                               NaN   
      3775                                               NaN   
      3776                                               NaN   
      3777                                               NaN   
      3778                                               NaN   
      3779                                  

In [29]:
df2 = pandas.concat((Out[i] for i in range(22,26)), axis='columns').dropna(how='all')